In [ ]:
import re
from datasets import load_dataset
from promptsource.templates import DatasetTemplates


TASK_INPUT_FILE = 'instruction/dataset_list.tsv'
OUTPUT_FILE = 'instruction/instructions.tsv'

data = []
has_no_templates = []
with open(TASK_INPUT_FILE) as fin:
    lines = fin.readlines()
for line in lines:
    splits = line.replace('\n', '').split('\t')
    if splits[1] == '':
        has_no_templates.append(splits[0])
        continue
    if splits[2] == '':
        splits[2] = None
    data.append(splits)


lines = []
for task in data:
    templates = DatasetTemplates(task[1], task[2]).templates

    dataset = load_dataset(task[1], task[2], split='train')
    example = dataset[0]

    line = '\t'.join([re.sub('\s+', ' ', t.apply(example)[0]) for t in templates.values()])
    lines.append(task[0] + '\t' + line)

with open(OUTPUT_FILE, 'w') as fout:
    fout.write('\n'.join(lines))

print('Cannot find templates for task:')
print(has_no_templates)